this script take key words as the input and return the cases that contain this input

In [1]:
import pandas as pd
import os
import numpy as np
import math

In [2]:
def process_string(string):
    # replace symbols
    output = string.replace(',',' , ').replace(':',' : ').replace('-'," - ")\
    .replace(';',' ; ').replace('?',' ? ').replace('. ',' . ').replace('  ',' ')
    # replace the white space in the start
    position = 0
    output = list(output)
    while True:
        if output[position] == ' ':
            output[position] = ''
            position += 1
        else:
            break
    output = "".join(output)
    return output

def indexes_for_duplicates_in_list(lst, item,lower = True):
    # in case the length of item > 1, combine every N element in the list together, N = len of item 
    len_item = len(item.split())
    new_lst = []
    for i in range(0,len(lst)-(len_item-1)):
        element = []
        for ii in range(0,len_item):
            element.append(lst[i + ii])
        new_lst.append(element)

    indexes = []
    if lower == True:
        indexes = [i for i,x in enumerate(new_lst) if  item in " ".join(x).lower() ]
    else:
        indexes = [i for i,x in enumerate(new_lst) if  item in " ".join(x)]
    
    result = []
    for i in indexes:
        for ii in range(0,len_item):
            result.append(i+ii)
    return result

In [3]:
def find_highlighted_sentence_w_keyword(case,keyword):
    # this function can also determine whether the case has the keyword
    has_keyword = 0
    
    report = case['Report text']
    # split report by sentence
    report_split = report.replace('\n','. ').split('. ')
    report_split = [i for i in report_split if i != ' ' and i != '']
    
    txt = ""
    highlight_sentence = []
    keyword_list = []
    for string in report_split:
        # find whether the key is included in each sentence
        keys = [key for key in keyword if key in process_string(string.lower())]
        if len(keys) != 0:
            has_keyword = 1
            # save keywords
            keyword_list.extend(keys)
            # find the indexes of the keyword in that sentence
            string = process_string(string)
            string_split = string.split()
            indexes_list = []
            for key in keys:
                index = indexes_for_duplicates_in_list(string_split,key)
                indexes_list.extend(index)
            indexes_list = sorted(indexes_list)
            
            # add this sentence and indexes into collection
            highlight_sentence.append((string,indexes_list))
            
            # write into text
            t = ""
            for i in range(0,len(string_split)):
                if i in indexes_list:
                    t = t + "!!" + string_split[i] + "!! "
                else:
                    t = t + string_split[i]+ ' '
            txt = txt + t +'.\n '
        
    keyword_list = list(dict.fromkeys(keyword_list))
    return has_keyword,keyword_list, highlight_sentence,txt
    

In [4]:
def make_highlighted_rtf_report(case,highlight_sentence,save_path,attach_full_report = True):
    # define rtf file name
    file_name = os.path.join(save_path,case['Patient_ID']+'.rtf')
    txt = "{\\rtf1 Patient_ID: "+case['Patient_ID']+"\line\line Highlight sentences: \line\line " 
    
    for h in highlight_sentence:
        string = h[0]
        indexes_list = h[1]
        string_split = string.split()
       
        # write into text
        t = ""
        for i in range(0,len(string_split)):
            if i in indexes_list:
                t = t + "\\b " + string_split[i] + " \\b0 "
            else:
                t = t + string_split[i]+ ' '
        txt = txt + t +'\line '
            
    # attach the full reports
    if attach_full_report == True:
        report = case['Report text']
        txt = txt + "\line Full reports:\line  " + report.replace('\n','\line ') + "}"
    
    output_file = open(file_name,'w')
    output_file.write(txt)
    output_file.close()
        

In [5]:
# define main folder
main_folder = '/Users/zhennongchen/Documents/Zhennong_CT_Data/Patient_Overview'

In [6]:
# make rtf files as well as a spreadsheet for highlight sentences
def main(data,keyword,make_rtf_file=0,make_spreadsheet=0,include_no_keyword_cases=1,print_text=False):
    
    # define save path
    os.makedirs(os.path.join(main_folder,'Case_search_list',year),exist_ok = True)
    save_path = os.path.join(main_folder,'Case_search_list',year,'rtf_files')
    os.makedirs(save_path,exist_ok = True)
    spreadsheet_name = year + '_Radiology_report_w_highlight.xlsx'
    column_list = ['Patient_ID','Keywords','Highlight text','Report text','Acession','Manufacturer','Model','Sex','Age','Protocol','Directories_Full','Directories_Function','Timeframes']

    count = 0
    data = data.fillna(0)
    spreadsheet = []
    for i in range(0,data.shape[0]):
        has_keyword = 0
        case = data.iloc[i]
        if case['Report text'] == 0: # no report text
            keyword_list = '';highlight_txt = ''
        else:

            has_keyword,keyword_list,highlight_sentence,highlight_txt = find_highlighted_sentence_w_keyword(case,keyword)

            if has_keyword == 1:
                count += 1
                if make_rtf_file == 1:
                    make_highlighted_rtf_report(case,highlight_sentence,save_path)
            else: # no keyword
                keyword_list = '';highlight_txt = ''
        if include_no_keyword_cases == 1:
            spreadsheet.append([case['Patient_ID'],keyword_list,highlight_txt,case['Report text'],case['Accession'],case['Manufacturer'],case['Model'],case['Sex'],\
                                case['Age'],case['Protocol'],case['Directories_Full'],\
                                case['Directories_Function'],case['Timeframes']])
        else:
            if has_keyword == 1:
                spreadsheet.append([case['Patient_ID'],keyword_list,highlight_txt,case['Report text'],case['Accession'],case['Manufacturer'],case['Model'],case['Sex'],\
                                case['Age'],case['Protocol'],case['Directories_Full'],\
                                case['Directories_Function'],case['Timeframes']])
            
    if print_text == True:
        print('finish')
        print(count)

    if make_spreadsheet == 1:
        df = pd.DataFrame(spreadsheet,columns = column_list)
        df.to_excel(os.path.join(os.path.dirname(save_path),spreadsheet_name),index = True)
    return count

keyword list:
0 hypoplastic
1 syndrome
2 congenital
3 chd
4 lvad
5 device
6 transcatheter
7 aortic stenosis
8 tavr
9 fibrillation
10 stenosis
11 coronary artery disease
12 lesion
13 ischemi
14 failure
15 thinning
16 infarct
17 hypokinesi
18 hypokinetic
19 akinesi
20 akinetic
21 dyskinesi
22 dyskinetic
23 hypertrophy
24 myopathy
25 wall motion
26 normal
27 function
28 ejection fraction

In [7]:
# define your keywords:
keyword = ['hypoplastic','syndrome','congenital','chd','lvad','device','transcatheter','aortic stenosis','tavr','fibrillation',\
           'stenosis','coronary artery disease','lesion','ischemi',\
           'failure','thinning','infarct','hypokinesi','hypokinetic','akinesi','akinetic','dyskinesi','dyskinetic','hypertrophy','myopathy',\
           'wall motion','normal','function','ejection fraction']
#keyword = ['cteph','embolism','lead','pacing']

In [33]:
keyword = ['stenosis','lesion']
data = pd.read_csv(os.path.join(main_folder,'Case_search_list','normal_candidates.csv')) 
count = 0
for i in range(0,data.shape[0]):
    case = data.iloc[i]
    keywords_list = case['Keywords']
    if 'aortic stenosis' in keywords_list:
        continue
        
    for key in keyword:
        if key in keywords_list:
            count += 1
    
        
print(count)
        

84


In [15]:
global year
year = '2019'

# load data
data = pd.read_csv(os.path.join(main_folder,year+'_Patient_Radiology_Records_Full.csv'))    
# only look for cases with funtional study
print('in total there are %d CT studies' % (data.shape[0]))
data = data[data['Function?'] == 'Yes']
print('there are %d CT studies with functional images' % (data.shape[0]))
# save rtf and excel:
main(data,keyword,1,1)
# stat
#print('%s stat by keyword search:' % (year))
#print('CHD patient: %d\nPatient with device: %d\nTAVR patient: %d\nPVA patient: %d\nPatient with stenosis mentioned: %d (aortic stenosis: %d)\nPatient with Heart failure: %d\nPatient with wall thinning or infarct mentioned: %d\nPatient with WMA: %d (hypo: %d, aki: %d, dys: %d)\nPatient with hypertrophy: %d\nPatient with wall motion mentioned: %d' %(main(data,keyword[0:4]),\
    #main(data,keyword[4:6]),main(data,keyword[6:9]),\
     #main(data,[keyword[9]]),main(data,keyword[10:14]),main(data,[keyword[7]]),main(data,[keyword[14]]),\
      #main(data,keyword[15:17]),main(data,keyword[17:23]),main(data,keyword[17:19]),\
      #main(data,keyword[19:21]),main(data,keyword[21:23]),main(data,keyword[23:25]),main(data,[keyword[25]])))

in total there are 636 CT studies
there are 357 CT studies with functional images
finish
98


98

In [8]:
year = '2017'

# load data
data = pd.read_csv(os.path.join(main_folder,year+'_Patient_Radiology_Records_Full.csv'))    
# only look for cases with funtional study
print('in total there are %d CT studies' % (data.shape[0]))
data = data[data['Function?'] == 'Yes']
print('there are %d CT studies with functional images' % (data.shape[0]))
# save rtf and excel:
main(data,keyword,1,1,True)
# stat
print('%s stat by keyword search:' % (year))
print('CHD patient: %d\nPatient with device: %d\nTAVR patient: %d\nPVA patient: %d\nPatient with stenosis mentioned: %d (aortic stenosis: %d)\nPatient with Heart failure: %d\nPatient with wall thinning or infarct mentioned: %d\nPatient with WMA: %d (hypo: %d, aki: %d, dys: %d)\nPatient with hypertrophy: %d\nPatient with wall motion mentioned: %d' %(main(data,keyword[0:4]),\
    main(data,keyword[4:6]),main(data,keyword[6:9]),\
      main(data,[keyword[9]]),main(data,keyword[10:14]),main(data,[keyword[7]]),main(data,[keyword[14]]),\
      main(data,keyword[15:17]),main(data,keyword[17:23]),main(data,keyword[17:19]),\
      main(data,keyword[19:21]),main(data,keyword[21:23]),main(data,keyword[23:25]),main(data,[keyword[25]])))

in total there are 88 CT studies
there are 60 CT studies with functional images
finish
58
2017 stat by keyword search:
CHD patient: 9
Patient with device: 3
TAVR patient: 19
PVA patient: 12
Patient with stenosis mentioned: 42 (aortic stenosis: 6)
Patient with Heart failure: 2
Patient with wall thinning or infarct mentioned: 3
Patient with WMA: 4 (hypo: 1, aki: 1, dys: 3)
Patient with hypertrophy: 9
Patient with wall motion mentioned: 2


In [9]:
global year
year = '2018'

# load data
data = pd.read_csv(os.path.join(main_folder,year+'_Patient_Radiology_Records_Full.csv'))    
# only look for cases with funtional study
print('in total there are %d CT studies' % (data.shape[0]))
data = data[data['Function?'] == 'Yes']
print('there are %d CT studies with functional images' % (data.shape[0]))
# save rtf and excel:
main(data,keyword,1,1,True)
# stat
print('%s stat by keyword search:' % (year))
print('CHD patient: %d\nPatient with device: %d\nTAVR patient: %d\nPVA patient: %d\nPatient with stenosis mentioned: %d (aortic stenosis: %d)\nPatient with Heart failure: %d\nPatient with wall thinning or infarct mentioned: %d\nPatient with WMA: %d (hypo: %d, aki: %d, dys: %d)\nPatient with hypertrophy: %d\nPatient with wall motion mentioned: %d' %(main(data,keyword[0:4]),\
    main(data,keyword[4:6]),main(data,keyword[6:9]),\
      main(data,[keyword[9]]),main(data,keyword[10:14]),main(data,[keyword[7]]),main(data,[keyword[14]]),\
      main(data,keyword[15:17]),main(data,keyword[17:23]),main(data,keyword[17:19]),\
      main(data,keyword[19:21]),main(data,keyword[21:23]),main(data,keyword[23:25]),main(data,[keyword[25]])))

in total there are 796 CT studies
there are 561 CT studies with functional images
finish
540
2018 stat by keyword search:
CHD patient: 57
Patient with device: 84
TAVR patient: 103
PVA patient: 154
Patient with stenosis mentioned: 325 (aortic stenosis: 59)
Patient with Heart failure: 12
Patient with wall thinning or infarct mentioned: 38
Patient with WMA: 36 (hypo: 29, aki: 14, dys: 8)
Patient with hypertrophy: 74
Patient with wall motion mentioned: 18


In [9]:
year = '2020'

# load data
data = pd.read_csv(os.path.join(main_folder,year+'_Patient_Radiology_Records_Full.csv'))    
# only look for cases with funtional study
print('in total there are %d CT studies' % (data.shape[0]))
data = data[data['Function?'] == 'Yes']
print('there are %d CT studies with functional images' % (data.shape[0]))
# save rtf and excel:
main(data,keyword,1,1,True)
# stat
print('%s stat by keyword search:' % (year))
print('CHD patient: %d\nPatient with device: %d\nTAVR patient: %d\nPVA patient: %d\nPatient with stenosis mentioned: %d (aortic stenosis: %d)\nPatient with Heart failure: %d\nPatient with wall thinning or infarct mentioned: %d\nPatient with WMA: %d (hypo: %d, aki: %d, dys: %d)\nPatient with hypertrophy: %d\nPatient with wall motion mentioned: %d' %(main(data,keyword[0:4]),\
    main(data,keyword[4:6]),main(data,keyword[6:9]),\
      main(data,[keyword[9]]),main(data,keyword[10:14]),main(data,[keyword[7]]),main(data,[keyword[14]]),\
      main(data,keyword[15:17]),main(data,keyword[17:23]),main(data,keyword[17:19]),\
      main(data,keyword[19:21]),main(data,keyword[21:23]),main(data,keyword[23:25]),main(data,[keyword[25]])))

in total there are 636 CT studies
there are 357 CT studies with functional images
finish
354
2020 stat by keyword search:
CHD patient: 53
Patient with device: 101
TAVR patient: 73
PVA patient: 56
Patient with stenosis mentioned: 229 (aortic stenosis: 63)
Patient with Heart failure: 15
Patient with wall thinning or infarct mentioned: 29
Patient with WMA: 51 (hypo: 47, aki: 14, dys: 4)
Patient with hypertrophy: 81
Patient with wall motion mentioned: 24


## make folders in NAS for all found patients

In [10]:
normal_patients = pd.read_csv(os.path.join(main_folder,'Case_search_list/normal_candidates.csv'))
abnormal_patients = pd.read_csv(os.path.join(main_folder,'Case_search_list/abnormal_candidates.csv'))

In [11]:
nas_drive = '/Volumes/McVeighLab/wip/zhennong'

In [14]:
print(normal_patients.shape[0])
for i in range(0,normal_patients.shape[0]):
    case = normal_patients.iloc[i]
    patient_id = case['Patient_ID']
    
    os.makedirs(os.path.join(nas_drive,'Normal',patient_id),exist_ok = True)
    os.makedirs(os.path.join(nas_drive,'Normal',patient_id,'img-dcm'),exist_ok = True)


198


In [16]:
D = os.listdir(os.path.join(nas_drive,'Normal'))
print(len(D))
count = 0
for o in D:
    if os.path.isdir(os.path.join(nas_drive,'Normal',o)):
        count += 1
    else:
        print(os.path.join(nas_drive,'Normal',o))
print(count)

199
/Volumes/McVeighLab/wip/zhennong/Normal/.DS_Store
198


## check whether the transfer is successful

In [19]:
for i in range(0,normal_patients.shape[0]):
    case = normal_patients.iloc[i]
    patient_id = case['Patient_ID']
    
    folder1 = os.path.join(nas_drive,'Normal',patient_id)
    if os.path.isdir(folder1) != 1:
        print(patient_id)
    
    D = os.listdir(os.path.join(folder1,'img-dcm'))
    if len(D) < 9:
        print(patient_id)

CVC1803021059
CVC1806211500
CVC1901041049
CVC1902011416
CVC1905281114
CVC1910071424
CVC2003041013


In [20]:
for i in range(0,abnormal_patients.shape[0]):
    case = abnormal_patients.iloc[i]
    patient_id = case['Patient_ID']
    
    folder1 = os.path.join(nas_drive,'Abnormal',patient_id)
    if os.path.isdir(folder1) != 1:
        print(patient_id)
    
    D = os.listdir(os.path.join(folder1,'img-dcm'))
    if len(D) < 9:
        print(patient_id)

CVC1802141345
CVC1803121651
CVC1804301009
CVC1805221523
CVC1801040932
CVC1901281242
CVC1902211408
CVC1903211114
CVC1904151124
CVC1905211108
CVC1906151814
CVC1908220904
CVC1908300959
CVC1911021733
CVC1901141121
CVC1902191419
CVC1904221514
CVC1910091025
CVC2003251150
CVC2004071136
CVC2004071506
CVC2004161410
CVC2006181459
